## Linear RegressionTask
- Load a data
- Profiling of data
- Write your analysis of data
- If there is some Na then impute
- Not normal, then handle it
- Check multicollinearity
- Build a model-linear regression
- save it 
- Model accuracy
- Ten test case
- Code in standard procedure

#### Data Set Link: https://archive.ics.uci.edu/ml/datasets/AI4I+2020+Predictive+Maintenance+Dataset

In [20]:
#important imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from pandas_profiling import ProfileReport
import seaborn as sns

In [49]:
#loading the data
df = pd.read_csv("ai4i2020.csv")

In [50]:
#head command to show first five record
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [7]:
#profile report from pandas_profiling

pf = ProfileReport(df)
pf.to_widgets()

Summarize dataset:   0%|          | 0/27 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### My Analysis from data:

- Total number of columns = 14 (6 numerical variables and 8 categorical variables).


- Total number of observation = 10000


- No duplicate instance of data present.


- UID is simply a identifier ranging from 1 to length of data. So we will drop it.


- Product ID has 10000 distict ID number,means we have 10000 different products. The most occuring product ID is starting with L means from low category.i.e 6000. Product ID is not going to contribute in model building, so we will drop it.


- We have total 3 type of product. L, M and H. L for low, M for Mdeium and H for high.


- Air Tem has min temp of 295 and max 304 in complete data set. It is normally distributed. Air tem has positive correlation with process temperature.


- Rotational Speed is little bit right skewed. which means maximum rotataional speed has occured less number of time.
- min rotational speed is 1168 rpm and maximum is 2886 rpm. we have to handle it while model creation.


- Torque values are normally distributed. Min torque is 3.8 Nm and maxium is 76.6 Nm. Torque has high negative correlation with Rotational speed.


- Machine Failure has 2 distinct values 0 and 1. 0 measn machine not failed and 1 means machine has failed. In this data set we have most instances belong to 0(not failed).According to this variable if we separate whole data points then our data set is imbalanced.


- We have total 5 kind of machine failure here in this data set.
- TWF(Tool Wear Failure)
- HDF(Heat Dissipationn Failure)
- PWF(Power Failure)
- OSF(overstrain Failure)
- RNF(Random Failure)

### Null values check

In [8]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64

No null values present in this data set

In [51]:
#dropping UDI, Product ID and Type

df = df.drop("UDI", axis = 1)
df = df.drop("Product ID", axis = 1)
df = df.drop("Type", axis = 1)

In [53]:
#correlation of Air Temperature with other variables
df.corr()["Air temperature [K]"]

Air temperature [K]        1.000000
Process temperature [K]    0.876107
Rotational speed [rpm]     0.022670
Torque [Nm]               -0.013778
Tool wear [min]            0.013853
Machine failure            0.082556
TWF                        0.009955
HDF                        0.137831
PWF                        0.003470
OSF                        0.001988
RNF                        0.017688
Name: Air temperature [K], dtype: float64

Air temperature has high correlation with Process Temperature and with others not showing very good correlation.

### Multicollinearity check

In [28]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [54]:
X = df.drop("Air temperature [K]", axis = 1)

In [55]:
X

,Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,308.6,1551,42.8,0,0,0,0,0,0,0
1,308.7,1408,46.3,3,0,0,0,0,0,0
2,308.5,1498,49.4,5,0,0,0,0,0,0
3,308.6,1433,39.5,7,0,0,0,0,0,0
4,308.7,1408,40.0,9,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9995,308.4,1604,29.5,14,0,0,0,0,0,0
9996,308.4,1632,31.8,17,0,0,0,0,0,0
9997,308.6,1645,33.4,22,0,0,0,0,0,0
9998,308.7,1408,48.5,25,0,0,0,0,0,0


In [58]:
# calculating VIF for each feature

vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

In [59]:
vif_data

,feature,VIF
0,Process temperature [K],776.333321
1,Rotational speed [rpm],381.026426
2,Torque [Nm],88.154518
3,Tool wear [min],4.031247
4,Machine failure,12.242962
5,TWF,2.444197
6,HDF,4.648895
7,PWF,3.657806
8,OSF,3.380730
9,RNF,1.003372


- Greater VIF denotes greater correlation. 
- Generally, a VIF above 5 indicates a high multicollinearity. 


- Here Process tem, Rotational speed, torque and Machine failure are showing multicollinearity.

In [60]:
X = X.drop("Process temperature [K]", axis = 1)

In [61]:
vif_data = pd.DataFrame()
vif_data["feature"] = X.columns

vif_data["VIF"] = [variance_inflation_factor(X.values, i)
                          for i in range(len(X.columns))]

In [62]:
vif_data

,feature,VIF
0,Rotational speed [rpm],10.345305
1,Torque [Nm],9.170284
2,Tool wear [min],4.005479
3,Machine failure,12.207100
4,TWF,2.438528
5,HDF,4.640521
6,PWF,3.533355
7,OSF,3.379303
8,RNF,1.003350


- Dropping Process temperature variable decreases multicollinearity.


- For our model building part we will drop Process temp.


### Model Building

In [63]:
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


In [64]:
df = df.drop("Process temperature [K]", axis = 1)

In [65]:
df.head()

,Air temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,298.1,1551,42.8,0,0,0,0,0,0,0
1,298.2,1408,46.3,3,0,0,0,0,0,0
2,298.1,1498,49.4,5,0,0,0,0,0,0
3,298.2,1433,39.5,7,0,0,0,0,0,0
4,298.2,1408,40.0,9,0,0,0,0,0,0


In [66]:
#separating input and output data from df data frame
x = df.drop("Air temperature [K]", axis = 1)
y = df["Air temperature [K]"]

In [68]:
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()

In [69]:
#Passing input and output data to linear model
linear_model.fit(x,y)

LinearRegression()

In [71]:
#Intercept of model
linear_model.intercept_

299.02596819566435

In [72]:
#Coeficients of model
linear_model.coef_

array([ 5.39161970e-04,  1.80759940e-03,  4.15885712e-04,  6.30170297e-02,
        1.98462086e-01,  2.62026470e+00, -1.65231179e-01, -1.10286560e-01,
        8.44692342e-01])

In [73]:
#Score of linear model 
linear_model.score(x,y)

0.02123051345747573

But dropping Process temperature giving worst accuracy even same instance of data.

- Let's try another model building with Process temperature

In [74]:
#loading the data
df = pd.read_csv("ai4i2020.csv")

In [75]:
df = df.drop("UDI", axis = 1)
df = df.drop("Product ID", axis = 1)
df = df.drop("Type", axis = 1)

In [76]:
#separating input and output data from df data frame
x = df.drop("Air temperature [K]", axis = 1)
y = df["Air temperature [K]"]

In [77]:
#Passing input and output data to linear model
linear_model.fit(x,y)

LinearRegression()

In [79]:
#Intercept of model
linear_model.intercept_

-64.20013022354334

In [80]:
#Coeficients of model
linear_model.coef_

array([ 1.17376384e+00,  1.90233804e-04,  2.67890316e-04,  8.45451276e-05,
       -9.00858666e-02,  2.01756471e-01,  1.77926496e+00,  1.41970075e-01,
       -4.04797285e-02, -5.72344840e-02])

In [78]:
#Score of linear model 
linear_model.score(x,y)

0.7756660136661062

This time score is far better than previous one.

### Saving the model

In [81]:
file = 'linear_reg.sav'
pickle.dump(linear_model,open(file,"wb"))

### Loading saved model for prediction using test cases

In [82]:
saved_linear_model = pickle.load(open(file,"rb"))

In [90]:
test_case = [[308.6,1551,42.8,0,0,0,0,0,0,0],[408.6,1661,52.8,0,1,0,1,0,0,0],[508.6,1888,62.9,0,0,0,0,0,0,0],
             [803.6,1111,24.8,0,1,0,0,0,0,1],[502.6,1751,42.8,0,0,0,0,0,0,0], [236.6,1771,93.8,0,0,0,0,0,0,0],
             [506.6,1774,63.8,0,0,0,0,0,0,0],[908.6,1651,85.8,0,0,0,0,0,0,0],[654.6,1931,85.8,0,0,0,0,0,0,0],
             [803.9,1663,92.8,0,1,0,0,0,0,1]]

In [96]:
try:
    for i in test_case:
        #making prediction
        print(f"Predcited Air Temp for this input {i} is {saved_linear_model.predict([i])}")
        
except Exception as e:
    print("error occured", e)

Predcited Air Temp for this input [308.6, 1551, 42.8, 0, 0, 0, 0, 0, 0, 0] is [298.32990779]
Predcited Air Temp for this input [408.6, 1661, 52.8, 0, 1, 0, 1, 0, 0, 0] is [417.41907506]
Predcited Air Temp for this input [508.6, 1888, 62.9, 0, 0, 0, 0, 0, 0, 0] is [533.1521683]
Predcited Air Temp for this input [803.6, 1111, 24.8, 0, 1, 0, 0, 0, 0, 1] is [879.10716117]
Predcited Air Temp for this input [502.6, 1751, 42.8, 0, 0, 0, 0, 0, 0, 0] is [526.07813866]
Predcited Air Temp for this input [236.6, 1771, 93.8, 0, 0, 0, 0, 0, 0, 0] is [213.87442546]
Predcited Air Temp for this input [506.6, 1774, 63.8, 0, 0, 0, 0, 0, 0, 0] is [530.78319507]
Predcited Air Temp for this input [908.6, 1651, 85.8, 0, 0, 0, 0, 0, 0, 0] is [1002.61875182]
Predcited Air Temp for this input [654.6, 1931, 85.8, 0, 0, 0, 0, 0, 0, 0] is [704.53600304]
Predcited Air Temp for this input [803.9, 1663, 92.8, 0, 1, 0, 0, 0, 0, 1] is [879.58251592]
